In [77]:
from time import sleep
from typing import Dict, Union
import uuid
import requests
import pandas as pd

from datetime import datetime, timedelta, date as date_type

import time

from requests import RequestException


def request_with_retries(requests_method, url,
                         max_retries=5, retry_delay=60, err_prefix="", **kwargs):
    status_codes = []
    for attempt in range(max_retries):
        try:
            response = requests_method(url, **kwargs)

            if response.status_code in [429, 500, 502, 503, 504, 404]:
                print(f"{err_prefix}Ошибка {response.status_code}. "
                      f"Попытка {attempt + 1} из {max_retries}. Ожидание {retry_delay} секунд...")
                status_codes.append(response.status_code)
                time.sleep(retry_delay)
            else:
                return response

        except RequestException as e:
            print(f"{err_prefix}Ошибка сети: {e}. Попытка {attempt + 1} из {max_retries}. Ожидание {retry_delay} секунд...")
            time.sleep(retry_delay)

    raise Exception(f"{err_prefix}Запрос не выполнен после {max_retries} попыток, status_codes={status_codes}.")



def fetch_wb_orders_data(marketplace_keys: Dict[str, str], date: Union[datetime, date_type]) -> pd.DataFrame:
    """
    Получает данные по остаткам через API Wildberries.

    :param marketplace_keys: Пары ключ-значение с seller_legal и api_key.
    """
    try:
        formatted_date = date.strftime("%Y-%m-%d")
    except ValueError as e:
        raise Exception(f"Неверный формат даты. Ожидается DD.MM.YYYY. {str(e)}")

    # Настройка URL и параметров API
    base_url = "https://statistics-api.wildberries.ru/api/v1/supplier/orders"
    params = {
        "dateFrom": formatted_date,
        "dateTo": formatted_date,
        "flag": 0
    }

    df_list = []
    for seller_legal, api_key in marketplace_keys.items():
        log_prefix = f"Заказы wb. {seller_legal}. "
        err_prefix = f"\033[91mWARNING: {log_prefix}\033[0m"
        headers = {
            "Authorization": api_key
        }
        response = request_with_retries(
            requests.get, url=base_url, headers=headers, params=params, err_prefix=err_prefix
        )
        if response.status_code != 200:
            raise Exception(f"{err_prefix}Ошибка при создании отчета: {response.status_code}, {response.text}")
        
        data = response.json()
        df = pd.DataFrame(data)
        if df.empty:
            continue
        if isinstance(date, datetime):
            date_from_dt = datetime.combine(date.date(), datetime.min.time())
        elif isinstance(date, date_type):
            date_from_dt = datetime.combine(date, datetime.min.time())
        date_to_dt = date_from_dt + timedelta(days=1) - timedelta(seconds=1)

        # Преобразуем столбец "date" в формат datetime для фильтрации
        df['date_datetime'] = pd.to_datetime(df['date'], format="%Y-%m-%dT%H:%M:%S")

        # Фильтруем строки, оставляя только те, которые попадают в диапазон
        df = df[(df['date_datetime'] >= date_from_dt) & (df['date_datetime'] <= date_to_dt)]

        # Удаляем временный столбец "date_datetime", чтобы вернуть данные в исходном формате
        df.drop(columns=['date_datetime'], inplace=True)

        # Обработка данных: добавляем юрлицо
        if seller_legal == "inter":
            df["legal_entity"] = "ИНТЕР"
        elif seller_legal == "ut":
            df["legal_entity"] = "АТ"
        elif seller_legal == "kravchik":
            df["legal_entity"] = "КРАВЧИК"
        else:
            raise Exception(f"Необработанный seller_legal={seller_legal}")

        if not df.empty:
            df_list.append(df)

    final_df = pd.concat(df_list, ignore_index=True)
    final_df = final_df.drop_duplicates()

    final_df['uuid'] = [str(uuid.uuid4()) for _ in range(len(final_df))]

    return final_df


In [11]:
config = {
}

In [78]:
import pytz
moscow_tz = pytz.timezone('Europe/Moscow')
date = datetime.now(moscow_tz)

In [87]:
date = date - timedelta(days=8)

In [88]:
date

datetime.datetime(2025, 1, 19, 1, 7, 20, 156393, tzinfo=<DstTzInfo 'Europe/Moscow' MSK+3:00:00 STD>)

In [89]:
df = fetch_wb_orders_data(config, date)

In [28]:
pd.set_option('display.max_columns', None)

In [90]:
df.shape

(439, 30)

In [58]:
df.shape

(351, 30)

In [66]:
# Удаляем дубликаты по ключевым столбцам
key_columns = ["supplierArticle", "nmId", "barcode", "warehouseName"]
if all(col in df.columns for col in key_columns):
    print(df.drop_duplicates(subset=key_columns).shape)

(132, 30)


In [62]:
df.iloc[0]

date                                2025-01-26T00:16:27
lastChangeDate                      2025-01-26T01:16:59
warehouseName                                  Коледино
warehouseType                                  Склад WB
countryName                                      Россия
oblastOkrugName           Центральный федеральный округ
regionName                           Ивановская область
supplierArticle                                H-GFC1-1
nmId                                          196426420
barcode                                   4673737630619
category                             Посуда и инвентарь
subject                         Контейнеры неполимерные
brand                                              AQRA
techSize                                              0
incomeID                                       24533892
isSupply                                          False
isRealization                                      True
totalPrice                                      

In [67]:
df[(df.supplierArticle == 'H-GFC1-1') & (df.nmId == 196426420) & (df.barcode == '4673737630619') & (df.warehouseName == 'Коледино')]

,date,lastChangeDate,warehouseName,warehouseType,countryName,oblastOkrugName,regionName,supplierArticle,nmId,barcode,category,subject,brand,techSize,incomeID,isSupply,isRealization,totalPrice,discountPercent,spp,finishedPrice,priceWithDisc,isCancel,cancelDate,orderType,sticker,gNumber,srid,legal_entity,uuid
0,2025-01-26T00:16:27,2025-01-26T01:16:59,Коледино,Склад WB,Россия,Центральный федеральный округ,Ивановская область,H-GFC1-1,196426420,4673737630619,Посуда и инвентарь,Контейнеры неполимерные,AQRA,0,24533892,False,True,4000,55,23,1386.0,1800.0,False,0001-01-01T00:00:00,Клиентский,29680637359,72195456643927540,8155522894378993990.0.0,ИНТЕР,820aebb9-bddd-4055-a7f6-52a5af974da4
1,2025-01-26T00:51:54,2025-01-26T01:16:59,Коледино,Склад WB,Россия,Центральный федеральный округ,Московская область,H-GFC1-1,196426420,4673737630619,Посуда и инвентарь,Контейнеры неполимерные,AQRA,0,24533892,False,True,4000,55,23,1386.0,1800.0,False,0001-01-01T00:00:00,Клиентский,29680637361,98081044779708183266,22901962603898555.0.0,ИНТЕР,94e3af98-560f-40ed-b44d-f673066b0b23
25,2025-01-26T09:04:14,2025-01-26T12:09:11,Коледино,Склад WB,Россия,Центральный федеральный округ,Курская область,H-GFC1-1,196426420,4673737630619,Посуда и инвентарь,Контейнеры неполимерные,AQRA,0,24533892,False,True,4000,55,23,1386.0,1800.0,False,0001-01-01T00:00:00,Клиентский,28873741250,2935692573975369782,5875834533532144125.3.0,ИНТЕР,13157fd3-a5b6-453c-8cbe-27624b05c931
26,2025-01-26T10:35:00,2025-01-26T12:09:12,Коледино,Склад WB,Россия,Центральный федеральный округ,Орловская область,H-GFC1-1,196426420,4673737630619,Посуда и инвентарь,Контейнеры неполимерные,AQRA,0,24533892,False,True,4000,55,23,1386.0,1800.0,False,0001-01-01T00:00:00,Клиентский,28873741254,9802411050593447078,15827136603916049.0.0,ИНТЕР,a8ee80f5-d994-4569-8b32-e1d7ffb345d2
57,2025-01-26T12:34:24,2025-01-26T14:46:03,Коледино,Склад WB,Россия,Центральный федеральный округ,Москва,H-GFC1-1,196426420,4673737630619,Посуда и инвентарь,Контейнеры неполимерные,AQRA,0,24533892,False,True,4000,55,23,1386.0,1800.0,False,0001-01-01T00:00:00,Клиентский,28873736240,92893299815163247596,37514657603919631.5.0,ИНТЕР,96d6cc97-a013-4243-8cb5-811bcd23e0be
58,2025-01-26T12:48:16,2025-01-26T14:46:04,Коледино,Склад WB,Россия,Центральный федеральный округ,Москва,H-GFC1-1,196426420,4673737630619,Посуда и инвентарь,Контейнеры неполимерные,AQRA,0,24533892,False,True,4000,55,23,1386.0,1800.0,False,0001-01-01T00:00:00,Клиентский,28873736238,95203544003502013232,11683313603920047.0.0,ИНТЕР,2ca78315-6b61-4274-9ec6-8677e9cd612e
81,2025-01-26T13:36:42,2025-01-26T15:54:36,Коледино,Склад WB,Россия,Центральный федеральный округ,Московская область,H-GFC1-1,196426420,4673737630619,Посуда и инвентарь,Контейнеры неполимерные,AQRA,0,24533892,False,True,4000,55,23,1386.0,1800.0,False,0001-01-01T00:00:00,Клиентский,28873303840,8339741639823547150,14680534603921500.1.1,ИНТЕР,8992bdfe-0563-42fa-b36b-746cce8b0e21
82,2025-01-26T13:36:42,2025-01-26T15:54:36,Коледино,Склад WB,Россия,Центральный федеральный округ,Московская область,H-GFC1-1,196426420,4673737630619,Посуда и инвентарь,Контейнеры неполимерные,AQRA,0,24533892,False,True,4000,55,23,1386.0,1800.0,False,0001-01-01T00:00:00,Клиентский,28873303842,8339741639823547150,14680534603921500.1.0,ИНТЕР,50fea3ec-1657-42eb-8f99-90e2f861e941
106,2025-01-26T15:03:40,2025-01-26T16:52:12,Коледино,Склад WB,Россия,Центральный федеральный округ,Москва,H-GFC1-1,196426420,4673737630619,Посуда и инвентарь,Контейнеры неполимерные,AQRA,0,24533892,False,True,4000,55,23,1386.0,1800.0,False,0001-01-01T00:00:00,Клиентский,28873719917,6611860476452218585,25489150103924109.5.0,ИНТЕР,c4a02b90-2724-4753-8de8-3d9cf12ad41b
129,2025-01-26T15:51:21,2025-01-26T19:26:00,Коледино,Склад WB,Россия,Центральный федеральный округ,Москва,H-GFC1-1,196426420,4673737630619,Посуда и инвентарь,Контейнеры неполимерные,AQRA,0,24533892,False,True,4000,55,23,1386.0,1800.0,False,0001-01-01T00:00:00,Клиентский,28873719697,5106431502163674176,4f51e839464d4

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351 entries, 0 to 350
Data columns (total 30 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             351 non-null    object 
 1   lastChangeDate   351 non-null    object 
 2   warehouseName    351 non-null    object 
 3   warehouseType    351 non-null    object 
 4   countryName      351 non-null    object 
 5   oblastOkrugName  351 non-null    object 
 6   regionName       351 non-null    object 
 7   supplierArticle  351 non-null    object 
 8   nmId             351 non-null    int64  
 9   barcode          351 non-null    object 
 10  category         351 non-null    object 
 11  subject          351 non-null    object 
 12  brand            351 non-null    object 
 13  techSize         351 non-null    object 
 14  incomeID         351 non-null    int64  
 15  isSupply         351 non-null    bool   
 16  isRealization    351 non-null    bool   
 17  totalPrice      

In [69]:
df.cancelDate.unique()

array(['0001-01-01T00:00:00', '2025-01-26T00:00:00'], dtype=object)